In [1]:
Output = '/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1015. Fire Sufficient Similarity/3. Analyses/2. Data Processing/Output'
cur_date = '021925'

library(missForest)
library(readxl)
library(tidyverse)
library(imputeLCMD)
library(factoextra)

# reading in files
ws_df = data.frame(Data = 'WS', read_excel("Input/Woodsmoke_Data_012825.xlsx", sheet = 2)) %>%
    select(-Sample_Number)
wf_df = data.frame(Data = 'WF', read_excel("Input/Wildfire_Data_012825.xlsx", sheet = 2)) %>%
    select(-Sample_Number)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: tmvtnorm

Loading required package: mvtnorm

Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Loading required package: stats4

Loading required package: gmm

Loading required package: sandwich

Loading required package: norm

This package has some major limitations
(for example, it does not work reliably when
the number of variables exceeds 30),
an

NOTES TO SELF:
- change 'replicate' to 'instance' and 'chemical class' to 'class'

In [2]:
head(ws_df)
head(wf_df)

,Data,HAWC_ID,Study,Replicate,Chemical_Class,Metric,DTXSID,Name,Value
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,11.3
2,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,2.2999999999999998
3,WS,1257056,McCarrick et al. 2024,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,1.66
4,WS,267140,Alfheim and Ramdahl 1984,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA
5,WS,1263480,Burnet et al. 1990,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA
6,WS,1263480,Burnet et al. 1990,NA,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA


,Data,HAWC_ID,Study,Replicate,Chemical_Class,Metric,DTXSID,Name,Value
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WF,1289821,Liang et al. 2021,1,PAH,Volume,NA,1-(10-methylanthracen-9-yl)ethanone,5
2,WF,1289821,Liang et al. 2021,1,PAH,Volume,DTXSID50176885,1-Acenaphthenone,1
3,WF,1289737,Campbell et al. 2024,1,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA
4,WF,1289739,Campos et al. 2019,1,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA
5,WF,1289739,Campos et al. 2019,NA,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA
6,WF,1289739,Campos et al. 2019,NA,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA


In [3]:
dim(ws_df)
dim(wf_df)

[1] 6607    9

[1] 4340    9

# Filter 1

Removing duplicate records for instances that weren't measured at all (MAR). 

In [4]:
`%notin%` <- Negate(`%in%`)

ws_df = ws_df %>%
    filter(Replicate %notin% NA)

wf_df = wf_df %>%
    filter(Replicate %notin% NA)

In [5]:
dim(ws_df)
dim(wf_df)

[1] 4203    9

[1] 2378    9

In [6]:
# combining dfs
combined_df = rbind(ws_df, wf_df)
head(combined_df)

,Data,HAWC_ID,Study,Replicate,Chemical_Class,Metric,DTXSID,Name,Value
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,11.3
2,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,2.2999999999999998
3,WS,1257056,McCarrick et al. 2024,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,1.66
4,WS,267140,Alfheim and Ramdahl 1984,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA
5,WS,1263480,Burnet et al. 1990,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA
6,WS,267091,Forchhammer et al. 2012,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA


In [7]:
length(unique(combined_df$Name))

[1] 192

192 unique variables.

# Variable Background Filters

Determining what variables have missing data for all variables delineated based on the dfs the data will be split into and analyzed. 

1. A variable (ie. `Name` which represents a chemical or metal) will be removed if < 50% of its experimental data points were either measurable concentrations and/or specified as non-detects. In other words, chemicals were excluded if they were NAs (not evaluated) across >= 50% of the samples.
2. The data needs to have at least one experimental (ie. measured or ND) value in both WS and WF data.

In [8]:
sample_type_presence_df = combined_df %>%
    # if the value isn't MAR count it as being present
    mutate(count = ifelse(Value != 'NA', 1, 0)) %>%
    # determining which have at least one experimental value within each sample type
    group_by(Data, Metric, Name) %>%
    # summing the number of experimental records for each variable
    reframe(data_group_count = sum(count))

head(sample_type_presence_df)

Data,Metric,Name,data_group_count
<chr>,<chr>,<chr>,<dbl>
WF,Volume,"1,2,4-Trimethylbenzene",2
WF,Volume,"1,3,5-Trimethylbenzene",2
WF,Volume,"1,3-Dihydroxynaphthalene",1
WF,Volume,"1,4-Dichloro-2-butene, cis",1
WF,Volume,"1,4-Dichlorobenzene",1
WF,Volume,"1,8-Dihydroxynaphthalene",1


In [9]:
dim(sample_type_presence_df)

sample_type_keep_df = sample_type_presence_df %>%
    filter(data_group_count > 0) 

dim(sample_type_keep_df)

[1] 449   4

[1] 333   4

In [10]:
# filtering the original df
filter1_df = inner_join(sample_type_keep_df[,1:3], combined_df)
head(filter1_df)

Joining with `by = join_by(Data, Metric, Name)`


Data,Metric,Name,HAWC_ID,Study,Replicate,Chemical_Class,DTXSID,Value
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>
WF,Volume,"1,2,4-Trimethylbenzene",1289926,Wang et al. 2024,1,VOC,DTXSID6021402,730
WF,Volume,"1,2,4-Trimethylbenzene",1306371,Ketcherside et al. 2024,1,VOC,DTXSID6021402,130
WF,Volume,"1,3,5-Trimethylbenzene",1289926,Wang et al. 2024,1,VOC,DTXSID6026797,330
WF,Volume,"1,3,5-Trimethylbenzene",1306371,Ketcherside et al. 2024,1,VOC,DTXSID6026797,110
WF,Volume,"1,3-Dihydroxynaphthalene",1289821,Liang et al. 2021,1,PAH,DTXSID40456587,6
WF,Volume,"1,4-Dichloro-2-butene, cis",1289926,Wang et al. 2024,1,VOC,DTXSID3027405,230


In [11]:
dim(combined_df)
dim(filter1_df)

[1] 6581    9

[1] 5218    9

Started with 6581 records, 1363 were removed, leaving 5218. 

Now that each sample type (WS of WF) has at least one experimental value, we'll see if there are at least 50% of data between the sample types.

In [12]:
variable_presence_df = filter1_df %>%
    # if the value isn't MAR count it as being present
    mutate(count = ifelse(Value != 'NA', 1, 0)) %>%
    group_by(Metric, Name) %>%
    # calculating the percentage of variables with data overall
    reframe(Variable_Presence_Percentage = (sum(count)/n()) * 100) %>%
    arrange(-Variable_Presence_Percentage)

# viewing data that passed the filter
keep_variables_df = variable_presence_df %>%
     filter(Variable_Presence_Percentage >= 50) %>%
     unique()

head(keep_variables_df)

Metric,Name,Variable_Presence_Percentage
<chr>,<chr>,<dbl>
Volume,"1,2,4-Trimethylbenzene",100
Volume,"1,3,5-Trimethylbenzene",100
Volume,"1,3-Dihydroxynaphthalene",100
Volume,"1,4-Dichloro-2-butene, cis",100
Volume,"1,4-Dichlorobenzene",100
Volume,"1,8-Dihydroxynaphthalene",100


In [13]:
# only keeping records that passed the background filter
filter2_df = inner_join(keep_variables_df[,1:2], filter1_df) %>%
    # metals don't have a DTXSID, so making that col their name
    mutate(DTXSID = ifelse(DTXSID != 'NA', DTXSID, Name)) 
    

head(filter2_df)

Joining with `by = join_by(Metric, Name)`


Metric,Name,Data,HAWC_ID,Study,Replicate,Chemical_Class,DTXSID,Value
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>
Volume,"1,2,4-Trimethylbenzene",WF,1289926,Wang et al. 2024,1,VOC,DTXSID6021402,730
Volume,"1,2,4-Trimethylbenzene",WF,1306371,Ketcherside et al. 2024,1,VOC,DTXSID6021402,130
Volume,"1,3,5-Trimethylbenzene",WF,1289926,Wang et al. 2024,1,VOC,DTXSID6026797,330
Volume,"1,3,5-Trimethylbenzene",WF,1306371,Ketcherside et al. 2024,1,VOC,DTXSID6026797,110
Volume,"1,3-Dihydroxynaphthalene",WF,1289821,Liang et al. 2021,1,PAH,DTXSID40456587,6
Volume,"1,4-Dichloro-2-butene, cis",WF,1289926,Wang et al. 2024,1,VOC,DTXSID3027405,230


In [14]:
dim(filter2_df)

[1] 4882    9

An additional 336 records were removed, leaving 4882.

# Second Variable Background Filter

In the intial filter, samples were combined. However, this time variables (ie. metal or chemical) will be split based on their `Metric` (ie. volume or weight) and then retained if that variable in found in both woodsmoke and wildfire samples within volume or weight samples.

In [15]:
split_filtered_df = filter2_df %>%
    group_by(Data, Metric) %>%
    group_split

split_ws_vol_df = split_filtered_df[[3]]
split_ws_weight_df = split_filtered_df[[4]]
split_wf_vol_df = split_filtered_df[[1]]
split_wf_weight_df = split_filtered_df[[2]]

In [16]:
# seeing how many unique variables are in each df and if they're consistent in each file
length(unique(split_ws_vol_df$Name))
length(unique(split_wf_vol_df$Name))
length(unique(split_ws_weight_df$Name))
length(unique(split_wf_weight_df$Name))

[1] 95

[1] 67

[1] 85

[1] 68

In [17]:
# they're not so first getting variables that are in weight or volume samples
consistent_wf_vol_df = split_wf_vol_df %>%
    filter(Name %in% unique(split_ws_vol_df$Name))
consistent_ws_vol_df = split_ws_vol_df %>%
    filter(Name %in% consistent_wf_vol_df$Name)
consistent_wf_weight_df = split_wf_weight_df %>%
    filter(Name %in% unique(split_ws_weight_df$Name))
consistent_ws_weight_df = split_ws_weight_df %>%
    filter(Name %in% consistent_wf_weight_df$Name)

length(unique(consistent_wf_vol_df$Name))
length(unique(consistent_ws_vol_df$Name))
length(unique(consistent_wf_weight_df$Name))
length(unique(consistent_ws_weight_df$Name))

[1] 32

[1] 32

[1] 35

[1] 35

There were 95, 67, 85 and 68 woodsmoke weight, woodsmoke volume, wildfire weight, and wildfire volume samples, respectively. 32 variables were common between volume samples and 35 were common between weight records and will be retained.

In [18]:
# recombining data
vol_df = rbind(consistent_ws_vol_df, consistent_wf_vol_df)
weight_df = rbind(consistent_ws_weight_df, consistent_wf_weight_df)

head(vol_df)

Metric,Name,Data,HAWC_ID,Study,Replicate,Chemical_Class,DTXSID,Value
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>
Volume,Acenaphthylene,WS,1263480,Burnet et al. 1990,1,PAH,DTXSID3023845,6187000
Volume,Acenaphthylene,WS,1263480,Burnet et al. 1990,2,PAH,DTXSID3023845,18890500
Volume,Acenaphthylene,WS,1263480,Burnet et al. 1990,3,PAH,DTXSID3023845,7806000
Volume,Acenaphthylene,WS,1263484,Leese et al. 1989,1,PAH,DTXSID3023845,1100000
Volume,Acenaphthylene,WS,1263484,Leese et al. 1989,2,PAH,DTXSID3023845,2800000
Volume,Acenaphthylene,WS,429445,Rajput 2010,1,PAH,DTXSID3023845,53500


# QRILC Imputation

Imputing non-detect data.

In [19]:
head(weight_df %>%
    filter(Value == 'ND'))

Metric,Name,Data,HAWC_ID,Study,Replicate,Chemical_Class,DTXSID,Value
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>
Weight,Fluoranthene,WS,1098462,Niu et al. 2023,3,PAH,DTXSID3024104,ND
Weight,Phenanthrene,WS,1098462,Niu et al. 2023,3,PAH,DTXSID6024254,ND
Weight,Phenanthrene,WS,914540,Verma et al. 2021,7,PAH,DTXSID6024254,ND
Weight,Phenanthrene,WS,914540,Verma et al. 2021,8,PAH,DTXSID6024254,ND
Weight,Ni,WS,822010,Farina et al. 2019,1,Metal,Ni,ND
Weight,Ni,WS,299223,Kasurinen et al. 2015,2,Metal,Ni,ND


Only the weight dataframe has non-detect values, which will be imputed using QRILC. However, its MAR data that will be imputed using random forest (RF) will be removed entirely from the dataset.

In [20]:
mar_weight_df = weight_df %>%
    filter(Value == 'NA')

preimputed_df = anti_join(weight_df, mar_weight_df) #%>%
    # creating a sample id col
    #unite(Sample_ID, HAWC_ID, Name, Replicate, sep = '_', remove = FALSE)
preimputed_df$Value = as.numeric(preimputed_df$Value)

# reordering
preimputed_df = preimputed_df[,c(3,1,4:8,2,9)]

head(preimputed_df)

Joining with `by = join_by(Metric, Name, Data, HAWC_ID, Study, Replicate,
Chemical_Class, DTXSID, Value)`
Warning message:
“NAs introduced by coercion”


Data,Metric,HAWC_ID,Study,Replicate,Chemical_Class,DTXSID,Name,Value
<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
WS,Weight,821855,Erlandsson et al. 2020,1,PAH,DTXSID2060383,"2,3-Dimethylnaphthalene",0.05
WS,Weight,821855,Erlandsson et al. 2020,2,PAH,DTXSID2060383,"2,3-Dimethylnaphthalene",0.05
WS,Weight,1257056,McCarrick et al. 2024,1,PAH,DTXSID2060383,"2,3-Dimethylnaphthalene",0.04
WS,Weight,821855,Erlandsson et al. 2020,1,PAH,DTXSID8074819,2-Methylchrysene,7.40
WS,Weight,821855,Erlandsson et al. 2020,2,PAH,DTXSID8074819,2-Methylchrysene,12.10
WS,Weight,1257056,McCarrick et al. 2024,1,PAH,DTXSID8074819,2-Methylchrysene,15.45


In [47]:
test = preimputed_df %>%
    filter(Name == 'Fluoranthene', HAWC_ID == '1098462')
test

test1 = preimputed_df %>%
    filter(Chemical_Class == 'Metal')

test2 = preimputed_df %>%
    filter(HAWC_ID == study_id[3])
head(test2)

Data,Metric,HAWC_ID,Study,Replicate,Chemical_Class,DTXSID,Name,Value
<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
WS,Weight,1098462,Niu et al. 2023,1,PAH,DTXSID3024104,Fluoranthene,0.21
WS,Weight,1098462,Niu et al. 2023,2,PAH,DTXSID3024104,Fluoranthene,0.18
WS,Weight,1098462,Niu et al. 2023,3,PAH,DTXSID3024104,Fluoranthene,NA


Data,Metric,HAWC_ID,Study,Replicate,Chemical_Class,DTXSID,Name,Value
<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
WS,Weight,267105,Danielsen et al. 2011,1,PAH,DTXSID4020878,2-Methylphenanthrene,10.0
WS,Weight,267105,Danielsen et al. 2011,2,PAH,DTXSID4020878,2-Methylphenanthrene,37.0
WS,Weight,267105,Danielsen et al. 2011,1,Metal,Cu,Cu,65.6
WS,Weight,267105,Danielsen et al. 2011,2,Metal,Cu,Cu,34.4
WS,Weight,267105,Danielsen et al. 2011,1,PAH,DTXSID0047741,Dibenzothiophene,36.0
WS,Weight,267105,Danielsen et al. 2011,2,PAH,DTXSID0047741,Dibenzothiophene,103.0


In [61]:
QRILC_imputation = function(dataset){
      # """
    # Creating a quantile normalization function to normalize each sample.
    # :param (input): exposed and unexposed (vehicle) dfs
    # :output: 1 quantile normalized df
    # """
    #ADD WORDS
    wider_dataset = dataset%>%#[,c(1:2,7,9)] %>%
        # removing these cols temporarily 
        select(-c("HAWC_ID", "Chemical_Class", "Name")) %>%
        pivot_wider(names_from = DTXSID, values_from = Value)
    
    index_of_last_variable = length(colnames(wider_dataset))

    # normalizing data since that what the QRILC function wants
    # had to pseudo log transform to prevent Inf values
    QRILC_prep = wider_dataset[,6:dim(wider_dataset)[2]] %>%
         mutate_all(., function(x) log10(x + 1)) %>%
         as.matrix()
                    
    imputed_QRILC_object = impute.QRILC(QRILC_prep, tune.sigma = 0.1)
    # QRILC_log10_df = data.frame(imputed_QRILC_object[1]) 
    
    # # converting back the original scale
    # QRILC_df = QRILC_log10_df %>%
    #     mutate_all(., function(x) 10^x - 1)
     
    # imputed_dataset = data.frame(cbind(unique(dataset[,1:8]), QRILC_df)) %>%
    #      pivot_longer(cols = 9:dim(wider_dataset)[2], names_to = "Variable", values_to = "Value")
    
    # return(imputed_dataset)
    return(wider_dataset)
}

In [62]:
QRILC_imputation(test1)

Data,Metric,Study,Replicate,B,Cu,K,Mg,Zn,Ni,⋯,Co,Si,Sr,Ti,Ba,Rb,Li,Bi,U,Th
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
WS,Weight,Nordin et al. 2015,1,308,336.00,35800.00,1806.00,7280.00,NA,⋯,NA,NA,NA,NA,NA,200.00,NA,4,NA,NA
WS,Weight,Nordin et al. 2015,2,268,196.00,9200.00,1776.00,2040.00,NA,⋯,NA,NA,NA,NA,NA,56.00,NA,NA,NA,NA
WS,Weight,Arif et al. 2017,1,NA,132.00,60136.00,12016.00,2996.00,39.00,⋯,4.00,29315.00,213,227.00,1110.00,NA,NA,NA,NA,NA
WS,Weight,Arif et al. 2017,2,NA,242.00,180744.00,35933.00,1140.00,96.00,⋯,2.90,57321.00,292,108.00,718.00,NA,NA,NA,NA,NA
WS,Weight,Corsini et al. 2013,1,NA,280.00,NA,NA,5300.00,20.00,⋯,NA,2230.00,60,NA,NA,1250.00,NA,NA,NA,NA
WS,Weight,Corsini et al. 2013,2,NA,470.00,NA,NA,15460.00,40.00,⋯,NA,9120.00,120,NA,NA,1290.00,NA,NA,NA,NA
WS,Weight,Danielsen et al. 2011,1,NA,65.60,NA,NA,1030.00,14.70,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
WS,Weight,Danielsen et al. 2011,2,NA,34.40,NA,NA,750.00,10.30,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
WS,Weight,Dilger et al. 2016,1,NA,4.10,69363.50,NA,4178.80,0.50,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [23]:
# # imputing within each study
study_id = unique(preimputed_df$HAWC_ID)

# imputed_df = data.frame()
# for (i in 1:length(study_id)){
#     filtered_preimputed_df = preimputed_df %>%
#         filter(HAWC_ID == study_id[i])
#     if(i ==1){
#     idk = QRILC_imputation(filtered_preimputed_df)
#     print(idk)

#     imputed_df = rbind(imputed_df, filtered_preimputed_df)
#         }
# }
# # calling fn
# #imputed_df = QRILC_imputation(test)

# head(imputed_df)

In [43]:
test2

Data,Metric,HAWC_ID,Study,Replicate,Chemical_Class,DTXSID,Name,Value
<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
WS,Weight,267105,Danielsen et al. 2011,1,PAH,DTXSID4020878,2-Methylphenanthrene,10.0
WS,Weight,267105,Danielsen et al. 2011,2,PAH,DTXSID4020878,2-Methylphenanthrene,37.0
WS,Weight,267105,Danielsen et al. 2011,1,Metal,Cu,Cu,65.6
WS,Weight,267105,Danielsen et al. 2011,2,Metal,Cu,Cu,34.4
WS,Weight,267105,Danielsen et al. 2011,1,PAH,DTXSID0047741,Dibenzothiophene,36.0
WS,Weight,267105,Danielsen et al. 2011,2,PAH,DTXSID0047741,Dibenzothiophene,103.0
WS,Weight,267105,Danielsen et al. 2011,1,Metal,Zn,Zn,1030.0
WS,Weight,267105,Danielsen et al. 2011,2,Metal,Zn,Zn,750.0
WS,Weight,267105,Danielsen et al. 2011,1,PAH,DTXSID3024104,Fluoranthene,359.0
